In [2]:
factorial = [1]
def precalc_factorials(n):
    while len(factorial) <= n:
        factorial.append(factorial[-1] * len(factorial))
    
    
def binom(n, k):
    # precalculate factorials for speed!!! not here as this function is called often
    # here n >= k, which is always true with correct i in number_to_simplex
    # and k >= 0, which is also always true with correct break in number_to_simplex
    return factorial[n] // factorial[k] // factorial[n - k]


def simplex_to_number(simplex, n):
    # simplex is an iterable consists of unique sorted integers from [0, n - 1]
    # no assertions for speed
    precalc_factorials(n)
    k = len(simplex)
    res = 0
    pr = 0
    for el in simplex:
        n -= 1
        k -= 1
        while pr < el:
            res += binom(n, k)
            n -= 1
            pr += 1
        pr += 1
    return res


def asserted_simplex_to_number(simplex, n):
    # simplex is an indexed iterable consists of unique sorted integers from [0, n - 1]
    assert all(simplex[i] < simplex[i + 1] for i in range(len(simplex) - 1)), f"simplex should be sorted and contains pairwise distinct values, got {simplex}"
    assert all(isinstance(el, int) for el in simplex), f"simplex elements should be integers, got {tuple(type(el) for el in simplex)}"
    assert isinstance(n, int) and n >= 0, f"n should be a non-negative integer, got {n}"
    assert len(simplex) == 0 or (simplex[0] >= 0 and simplex[-1] < n), f"simplex elements should be indexed from 0 to {n-1} inclusively, got range [{simplex[0]}, {simplex[-1]}]"
    return simplex_to_number(simplex, n)

    
def number_to_simplex(n, k, i):
    # n >= k
    # i is an integer from [0, binom(n, k) - 1]
    # no assertions for speed
    precalc_factorials(n)
    res = []
    for j in range(n):
        cur = binom(n - j - 1, k - 1)
        if cur <= i:
            i -= cur
        else:
            res.append(j)
            k -= 1
            if k == 0:
                break
    return res


def asserted_number_to_simplex(n, k, i):
    # n >= k
    # i is an integer from [0, binom(n, k) - 1]
    precalc_factorials(n)  # binom in assertions
    assert isinstance(n, int) and isinstance(k, int) and n >= k >= 0, f"n and k should be non-negative integers, which satisfies n >= k, got {n=}, {k=}"
    assert isinstance(i, int) and i >= 0 and i < binom(n, k), f"i should be a non-negative integer from 0 to {binom(n, k) - 1} inclusively, got {i}"
    return number_to_simplex(n, k, i)

In [3]:
# tests
from itertools import combinations


def test_simplex_to_number(simplex, n):
    k = len(simplex)
    ret = asserted_simplex_to_number(simplex, n)
    assert isinstance(ret, int), f"simplex_to_number function should return integer, got {type(ret)}"
    ans = next((i for i, s in enumerate(combinations(range(n), k)) if s == tuple(simplex)), None)
    assert ans is not None, f"assertions in asserted_simplex_to_number are not full: {simplex} is not a combination (wrong argument)"
    assert ans == ret, f"wrong answer: got {ret}, but should be {ans}"
    return ans


def test_number_to_simplex(n, k, i):
    ret = asserted_number_to_simplex(n, k, i)
    ans = list(combinations(range(n), k))[i]  # binom(n, k) should not be large here! (but can be in number_to_simplex)
    assert len(ret) == k, f"wrong length: got {ret} with length {len(ret)}, should be {ans} with length {k}"
    cmp = test_simplex_to_number(ret, n)
    assert i == cmp, f"wrong answer: got {cmp}-th simplex ({ret}), but should be {i}-th ({ans})"
    return ret

In [4]:
test_simplex_to_number((0, 1, 2), 3)
test_simplex_to_number(tuple(), 0)
test_simplex_to_number([], 5)
test_simplex_to_number((1, 2), 3)
test_simplex_to_number((0,), 1)
test_simplex_to_number((9, 14, 18, 21, 37), 53)

1826807

In [11]:
test_number_to_simplex(5, 3, 9)
test_number_to_simplex(5, 3, 0)
test_number_to_simplex(5, 0, 0)
test_number_to_simplex(0, 0, 0)
test_number_to_simplex(19, 8, 10000)

[0, 1, 5, 7, 9, 10, 12, 18]

In [7]:
from random import randint
for _ in range(1000):
    n = randint(0, 20)
    k = randint(0, n)
    i = randint(0, len(list(combinations(range(n), k))) - 1)
    assert i == test_simplex_to_number(test_number_to_simplex(n, k, i), n)
print("OK")

OK
